<a href="https://colab.research.google.com/github/juharrais/visao_computacional/blob/main/SM_single_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nome: Juliana Arrais
# Segmentation Models Doc
# https://github.com/qubvel/segmentation_models

In [ ]:
#Libs & Configs
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#Lib installation
!pip3 install -r "gdrive/Shareddrives/Nuvens/requirements_keras/requirements.txt" --use-deprecated=legacy-resolver -q

     |████████████████████████████████| 129 kB 12.5 MB/s 
     |████████████████████████████████| 98 kB 5.9 MB/s 
     |████████████████████████████████| 727 kB 51.5 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 1.6 MB 45.2 MB/s 
     |████████████████████████████████| 9.7 MB 19.9 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 283 kB 50.7 MB/s 
     |████████████████████████████████| 10.7 MB 30.0 MB/s 
     |████████████████████████████████| 88 kB 6.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 50.8 MB/s 
     |████████████████████████████████| 402 kB 44.8 MB/s 
     |████████████████████████████████| 214 kB 49.3 MB/s 
     |████████████████████████████████| 1.0 MB 31.2 MB/s 
     |████████████████████████████████| 1.6 MB 42.2 MB/s 
     |█████

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import CSVLogger

# from keras.utils import normalize
import os
import pandas as pd
from requests import get

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["SM_FRAMEWORK"] = "tf.keras"


import segmentation_models as sm
sm.set_framework('tf.keras')
from segmentation_models.utils import set_trainable
import albumentations as A


import glob
import cv2
import numpy as np
import random

from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

In [ ]:
SIZE_X = 1024
SIZE_Y = SIZE_X
n_classes = 5
IMG_CHANNELS = 3 # Grayscale = 1 || RGB = 3

ALL_CLASSES = [
                "Arvore",                  #0
                "Estratocumuliformes",     #1
                "Estratiformes",           #2
                "Cirriformes",             #3
                "Cumuliformes",            #4
                
            ]

CLASSES = [
                "Arvore",                  #0
                "Estratocumuliformes",     #1
                "Estratiformes",           #2
                "Cirriformes",             #3
                "Cumuliformes",            #4
            ]

seed = 99
np.random.seed(seed)
random.seed(seed)

DATA_DIR = '/content/gdrive/Shareddrives/Nuvens/datasets/Albedo(merged classes)_001 - 997 images'
MODELS_DIR = 'gdrive/Shareddrives/Nuvens/trained_models/keras/'

notebook_filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
MODELS_DIR = MODELS_DIR + notebook_filename+'/'

RESULTS_DIR = MODELS_DIR + 'results/'

x_train_dir = os.path.join(DATA_DIR, 'images')
y_train_dir = os.path.join(DATA_DIR, 'labels')

x_valid_dir = os.path.join(DATA_DIR, 'valid_images')
y_valid_dir = os.path.join(DATA_DIR, 'valid_labels')

x_test_dir = os.path.join(DATA_DIR, 'test_images')
y_test_dir = os.path.join(DATA_DIR, 'test_labels')

In [ ]:
#Dataset class
# classes for data loading and preprocessing
class Dataset:
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = [
                "Arvore",                  #0
                "Estratocumuliformes",     #1
                "Estratiformes",           #2
                "Cirriformes",             #3
                "Cumuliformes",            #4
            ]
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            SIZE_X=SIZE_X,
            SIZE_Y=SIZE_Y
    ):
        self.ids = sorted(os.listdir(images_dir))
        self.ids_masks = sorted(os.listdir(masks_dir))
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids_masks]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing

        self.SIZE_X = SIZE_X
        self.SIZE_Y = SIZE_Y

        self.all_images = self._read_all_images()
        self.all_masks = self._read_all_masks()
    
    def _read_all_images(self):
        resized_images = []
        for current_img in self.images_fps:
            img = cv2.imread(current_img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (SIZE_Y, SIZE_X))
            resized_images.append(img)
        return resized_images

    def _read_all_masks(self):
        resized_masks = []
        for current_img in self.masks_fps:
            mask = cv2.imread(current_img, 0)
            mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)
            resized_masks.append(mask)
        return resized_masks

    def __getitem__(self, i):
        
        # read data
        if len(self.all_images) > 0:
            image = self.all_images[i]
        else:
            image = cv2.imread(self.images_fps[i])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if len(self.all_masks) > 0:
            mask = self.all_masks[i]
        else:
            mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cumulus, stratus)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)

In [ ]:
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

In [ ]:
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return tuple(batch)
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [ ]:
# helper function for data visualization
def visualize(cmap=None, img_id=-1, dir=None, figsize=(18, 8), **images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=figsize)
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, cmap=cmap)

        if cmap:
            plt.clim(0, n_classes-1)
            clb = plt.colorbar(shrink=0.55)
            clb.set_ticks(range(n_classes))
            clb.ax.tick_params(labelsize=10)
            clb.ax.set_yticklabels(ALL_CLASSES)
            clb.ax.set_visible(i == 2)
    # plt.show()
    image_name = 'Experiment #'+str(img_id)+'.png'
    plt.savefig(dir+image_name)

In [ ]:
#data augmentation
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

# define heavy augmentations
def get_training_augmentation(resize_to=(SIZE_X,SIZE_Y)):
    train_transform = [

        A.HorizontalFlip(p=1),


        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightnessContrast(p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),

      

        A.OneOf(
            [
                A.RandomBrightnessContrast(p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(SIZE_X, SIZE_Y)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [ ]:
#@title
# # Visualize the data
# # "Arvore": [1],
# # "Estratocumuliformes": [2],
# # "Estratiformes": [3],
# # "Cirriformes": [4],
# # "Cumuliformes": [5],



# classes = [
#             "Arvore",                   #0
#             "Estratocumuliformes",      #1
#             "Estratiformes",            #2
#             "Cirriformes",              #3
#             "Cumuliformes",             #4
#         ]
# dataset = Dataset(x_train_dir, y_train_dir, classes=classes)

# image, mask = dataset[0] # get some sample
# visualize(
#     image=image, 
#     Arvore=mask[..., 0].squeeze(),
#     # Estratocumuliformes=mask[..., 1].squeeze(),
#     # Estratiformes=mask[..., 2].squeeze(),
#     # Cirriformes=mask[..., 3].squeeze(),
#     # Cumuliformes=mask[..., 4].squeeze(),
#     
# )



# # "Arvore": [1],
# # "Estratocumuliformes": [2],
# # "Estratiformes": [3],
# # "Cirriformes": [4],
# # "Cumuliformes": [5],



# classes = [
#             "Arvore",                   #0
#             "Estratocumuliformes",      #1
#             "Estratiformes",            #2
#             "Cirriformes",              #3
#             "Cumuliformes",             #4
#         ]

# # Lets look at augmented data we have
# dataset = Dataset(x_train_dir, y_train_dir, classes=classes, augmentation=get_training_augmentation())

# image, mask = dataset[0] # get some sample
# visualize(
#     image=image, 
#     Arvore=mask[..., 0].squeeze(),
#     # Estratocumuliformes=mask[..., 1].squeeze(),
#     # Estratiformes=mask[..., 2].squeeze(),
#     # Cirriformes=mask[..., 3].squeeze(),
#     # Cumuliformes=mask[..., 4].squeeze(),
#    
# )

In [ ]:
#IoU Class Metric
class ClassIoU(keras.metrics.MeanIoU):
    """Computes the class-specific Intersection-Over-Union metric.

    IOU is defined as follows:
    IOU = true_positive / (true_positive + false_positive + false_negative).
    The predictions are accumulated in a confusion matrix, weighted by
    `sample_weight` and the metric is then calculated from it.

    If `sample_weight` is `None`, weights default to 1.
    Use `sample_weight` of 0 to mask values.

    Args:
    class_idx: The index of the the class of interest
    one_hot: Indicates if the input is a one_hot vector as in CategoricalCrossentropy or if the class indices
        are used as in SparseCategoricalCrossentropy or MeanIoU
    num_classes: The possible number of labels the prediction task can have.
        This value must be provided, since a confusion matrix of dimension =
        [num_classes, num_classes] will be allocated.
    name: (Optional) string name of the metric instance.
    dtype: (Optional) data type of the metric result.
    """
    def __init__(self, class_idx, one_hot, num_classes, name=None, dtype=None):
        super().__init__(num_classes, name, dtype)
        self.one_hot = one_hot
        self.class_idx = class_idx

    def result(self):
        sum_over_row = tf.cast(
            tf.reduce_sum(self.total_cm, axis=0), dtype=self._dtype)
        sum_over_col = tf.cast(
            tf.reduce_sum(self.total_cm, axis=1), dtype=self._dtype)
        true_positives = tf.cast(
            tf.linalg.diag_part(self.total_cm), dtype=self._dtype)

        # sum_over_row + sum_over_col =
        #     2 * true_positives + false_positives + false_negatives.
        denominator = sum_over_row[self.class_idx] + sum_over_col[self.class_idx] \
            - true_positives[self.class_idx]

        # The mean is only computed over classes that appear in the
        # label or prediction tensor. If the denominator is 0, we need to
        # ignore the class.
        num_valid_entries = tf.reduce_sum(
            tf.cast(tf.not_equal(denominator, 0), dtype=self._dtype))

        iou = tf.math.divide_no_nan(true_positives[self.class_idx], denominator)

        return tf.math.divide_no_nan(
            tf.reduce_sum(iou, name='mean_iou'), num_valid_entries)

    def update_state(self, y_true, y_pred, sample_weight=None):
        if self.one_hot:
            return super().update_state(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1), sample_weight)
        else:
            return super().update_state(y_true, y_pred, sample_weight)

In [ ]:
BATCH_SIZE = 32 
BACKBONE = 'resnet18'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
#dataloaders
# Dataset for train images

train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    classes=CLASSES, 
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

# Dataset for validation images
valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    classes=CLASSES, 
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

print(train_dataloader[0][0].shape)
print(valid_dataloader[0][1].shape)

ValueError: ignored

In [ ]:
EPOCHS = 250

#### Activation Function ####
n_classes = len(CLASSES) + 1
activation = 'softmax'

#### Create Model ####
model = sm.Linknet(BACKBONE, classes=n_classes, activation=activation, encoder_freeze=True)
model._name = 'Linknet' # Alterar de acordo com o modelo acima


optim = keras.optimizers.Adam()

#### Loss ####
total_loss = sm.losses.categorical_focal_dice_loss

#### Metrics ####
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

#### Model Compilation ####
model.compile(optim, total_loss, metrics)
# model.summary()

In [ ]:
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)

if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)

model_name = MODELS_DIR + total_loss.name + '__' + model.name + '__' + BACKBONE + '__' + str(EPOCHS) + 'epochs__' +str(SIZE_X)+'x'+str(SIZE_Y)+'.h5'

RESULTS_DIR_MODEL = RESULTS_DIR + model_name.split('/')[-1][:-3] + '/'
if not os.path.exists(RESULTS_DIR_MODEL):
    os.mkdir(RESULTS_DIR_MODEL)

if os.path.isfile(model_name):
    model.load_weights(model_name)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(model_name, save_weights_only=True, save_best_only=True, monitor='val_loss', mode='min'),
    keras.callbacks.ReduceLROnPlateau(factor=0.02, patience=2, min_lr=0.00001, monitor='val_loss'),
    CSVLogger(RESULTS_DIR_MODEL+'training.log', separator=',', append=True)
    
]

model_name

In [ ]:
#model Pre-training
model.fit(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs=int(EPOCHS*0.1), 
    callbacks=callbacks, 
    validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader)
)

In [ ]:
#model Training
### Unfreezing Layers ###
for layer in model.layers:
    layer.trainable = True

### Training ###
history = model.fit(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs=EPOCHS, 
    initial_epoch=0,
    callbacks=callbacks, 
    validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader)
)

In [ ]:
#Training Evolution
history = pd.read_csv(RESULTS_DIR_MODEL+'training.log', sep=',')

#Plot training & validation iou_score values
plt.figure(figsize=(30, 10))
plt.plot(history['iou_score'])
plt.plot(history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

image_name = 'iou_score.png'
plt.savefig(RESULTS_DIR_MODEL+image_name)

plt.figure(figsize=(30, 10))
plt.plot(history['f1-score'])
plt.plot(history['val_f1-score'])
plt.title('Model f1-score')
plt.ylabel('f1-score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

image_name = 'f1_score.png'
plt.savefig(RESULTS_DIR_MODEL+image_name)

# Plot training & validation loss values
plt.figure(figsize=(30, 10))
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

image_name = 'loss.png'
plt.savefig(RESULTS_DIR_MODEL+image_name)


In [ ]:
#Machine Specs & Training History & Model Summary
# GPU info
from tensorflow.python.client import device_lib
with open(RESULTS_DIR_MODEL+"gpu_specs.txt", "w") as text_file:
    text_file.write(str(device_lib.list_local_devices()))

# CPU info
import cpuinfo
with open(RESULTS_DIR_MODEL+"cpu_specs.txt", "w") as text_file:
    text_file.write(str(cpuinfo.get_cpu_info()))

# RAM info
import psutil
with open(RESULTS_DIR_MODEL+"ram_specs.txt", "w") as text_file:
    text_file.write(str(psutil.virtual_memory()))

# Model Summary
def summary_to_file(s):
    with open(RESULTS_DIR_MODEL+'model_summary.txt','a+') as f:
        print(s, file=f)
model.summary(print_fn = summary_to_file)

In [ ]:
#Test Dataloader
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=CLASSES, 
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

test_dataloader = Dataloder(test_dataset, batch_size=32, shuffle=False)

In [ ]:
#Load best weights only
model.load_weights(model_name)

#Save overall test results
scores = model.evaluate(test_dataloader)

with open(RESULTS_DIR_MODEL+"overall_scores.txt", "w") as text_file:
    text_file.write(str("Loss: {:.5}".format(scores[0])))
    print("Loss: {:.5}".format(scores[0]))

    for metric, value in zip(metrics, scores[1:]):
        text_file.write('\n')
        text_file.write(str("Mean {}: {:.5}".format(metric.__name__, value)))
        print("Mean {}: {:.5}".format(metric.__name__, value))

In [ ]:
#sabe images for qualitative analysis
# n = 20 # Quantidade de imagens de teste para visualização
# ids = random.sample(range(len(test_dataset)), n) # Amostra aleatória de n imagens de teste
ids = range(len(test_dataset))
img_index = 1

for i in ids:
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image)
    
    visualize(
        cmap='jet',
        img_id=img_index,
        dir=RESULTS_DIR_MODEL,
        image=denormalize(image.squeeze()),
        gt_mask=np.argmax(gt_mask, axis = 2),
        pr_mask=np.argmax(pr_mask, axis = 3).squeeze()
    )
    img_index += 1